# Stream Ciphers

In [1]:
def sum(a, b):
    # Suma modulo 2^32
    return (a + b) & ((1 << 32) - 1)

In [91]:
def str_to_hex(string):
    # I copy this from StackOverFlow
    nchars = len(string)
    return hex(sum(ord(string[byte])<<8*(nchars-byte-1) for byte in range(nchars)))

In [105]:
def hex_to_str(hex_string):
    hex_int = int(hex_string, 16)    
    result = ""
    while hex_int > 0:
        byte = hex_int & 0xFF
        result = chr(byte) + result
        hex_int >>= 8
    
    return result

# Salsa

### Pad

In [108]:
# constant word (32 bits each)
# "Expand 32-byte k"
c = [str_to_hex("expa"[::-1]),
     str_to_hex("nd 3"[::-1]),
     str_to_hex("2-by"[::-1]),
     str_to_hex("te k"[::-1])]
c

['0x61707865', '0x3320646e', '0x79622d32', '0x6b206574']

In [84]:

def pad(s, j, n):
    """A padding function that combies a 256 bits seed s with a 64 bit counter j (j0, j1 of 32 bits) and a b4 bit nonce (n0, n1 of 32 bits) to form a 512 bit block denoted x0, ..., x15 of 32 bits
    
    Args:
        s (array[hex]): Seed (256 bits, 7 words of 32 bits)
        j (array[hex]): counter (64 bits)
        n (array[hex]): Nonce (64 bits)

    Returns:
        array[hex]: 512 bits (16 words 32 bits)
    """

    return [
        c[0], s[0], s[1], s[2],
        s[3], c[1], n[0], n[1],
        j[0], j[1], c[2], s[4],
        s[5], s[6], s[7], c[3]
    ]

### Permutation $\pi$

Interating a simple permutation a fized number of times

In [85]:
def rot(w, r): 
    """Rotate lest for 32 bits

    Args:
        w (hex): word to rotate
        r (hex): I dont remember

    Returns:
        _type_: _description_
    """

    mask = 0xffffffff
    return ((w << r) & mask) | (w >> (32 - r)) 

In [86]:
def QR_salsa(a, b, c, d): 
    # Quater round
    b = b ^ rot(sum(a, d), 7)
    c = c ^ rot(sum(b, a), 9)
    d = d ^ rot(sum(c, b), 13)
    a = a ^ rot(sum(d, c), 18)
    
    return a, b, c, d

In [8]:
ROUNDS = 20

def perm_salsa(x): 
    """Permutation 

    Args:
        x (_type_): 512 bits (16 words 32 bits)

    Returns:
        _type_: 512 bits (16 words 32 bits)
    """
    
    i = x.copy()
    
    for _ in range(0, ROUNDS, 2): 
        # Odd round
        i[0], i[4], i[8], i[12] = QR_salsa(i[0], i[4], i[8], i[12]) # Col1
        i[5], i[9], i[13], i[1] = QR_salsa(i[5], i[9], i[13], i[1]) # Col2
        i[10], i[14], i[2], i[6] = QR_salsa(i[10], i[14], i[2], i[6]) # Col3
        i[15], i[3], i[7], i[11] = QR_salsa(i[15], i[3], i[7], i[11]) # Col4
        # Even round 
        i[0], i[1], i[2], i[3] = QR_salsa(i[0], i[1], i[2], i[3]) # Row1
        i[5], i[6], i[7], i[4] = QR_salsa(i[5], i[6], i[7], i[4]) # Row2
        i[10], i[11], i[8], i[9] = QR_salsa(i[10], i[11], i[8], i[9]) # Row3
        i[15], i[12], i[13], i[14] = QR_salsa(i[15], i[12], i[13], i[14]) # Row 4
    return i

### Generador psudoaletorio $G$

In [107]:
s = str_to_hex("*Thirty-two byte (256 bits) key*")

### Salsa 

## ChaCha

In [51]:
def QR_chacha(a, b, c, d):
    a = sum(a, b)
    d = d ^ a 
    d = rot(d, 16)

    c = sum(c, d)
    b = b ^ c
    b = rot(b, 12)

    a = sum(a, b)
    
    return a, b, c, d